### #2

In [16]:
# This cell contains what we need for both subitems of 2

P = []

# Reading inputs from the text file
with open('seg_ls.txt', 'r') as file:
    n = int(file.readline())
    
    for i in range(n):
        x, y = [float(x) for x in (file.readline()).split()]
        P.append((x,y))

P.sort()
        
# Creating the prefix-sum arrays for the summations
x_running = 0
y_running = 0
xy_running = 0
x2_running = 0
y2_running = 0

# We add zeroes so arr[i] denotes the sum from 1 to i
xs = [0]
ys = [0]
xys = [0]
x2s = [0]
y2s = [0]

for p in P:
    x_running += p[0]
    y_running += p[1]
    xy_running += p[0]*p[1]
    x2_running += p[0]**2
    y2_running += p[1]**2
    
    xs.append(x_running)
    ys.append(y_running)
    xys.append(xy_running)
    x2s.append(x2_running)
    y2s.append(y2_running)

# Defining functions to solve for aij, bij, and Lij
def sum_of(arr, i, j):
    return arr[j] - arr[i-1]    
    
def a(i,j): #intercept
    return (sum_of(ys,i,j) - b(i,j)*sum_of(xs,i,j))/(j-i+1)

def b(i,j): #slope
    return ((j-i+1)*sum_of(xys,i,j) - sum_of(xs,i,j)*sum_of(ys,i,j))/((j-i+1)*sum_of(x2s,i,j) - (sum_of(xs,i,j))**2)

def L(i,j):
    return (j-i+1)*(a(i,j)**2) + 2*a(i,j)*b(i,j)*sum_of(xs,i,j) + (b(i,j)**2)*sum_of(x2s,i,j) - 2*a(i,j)*sum_of(ys,i,j) - 2*b(i,j)*sum_of(xys,i,j) + sum_of(y2s,i,j)

a)

In [17]:
memo = dict()

best_choice = dict()
# Let f(j,k) be the minimum sum of squared errors obtained by using k lines to cover p_1, ..., p_j
def f(j,k):
    if (j,k) in memo:
        return memo[j,k]
    else:
        
        # If we have one line left, just get the error for the remaining points
        if k == 1:
            answer = L(1,j)
        
        # If not, get the minimum error when conducting linear regression on points i to j and 
        # modeling points 1,...,i-1 with k-1 lines
        else:
            answer = float('inf')
            
            for i in range(2*k-1, j):
                penalty = L(i,j) + f(i-1, k-1)
                
                if penalty < answer:
                    answer = penalty
                    choice = i
                
                # best_choice will store the optimal split for each j,k
                best_choice[j,k] = (choice, a(choice,j), b(choice,j))
            
        memo[j,k] = answer
        return memo[j,k]
    
print(f(n, 3))
print(best_choice)

294090.4364505969
{(4, 2): (3, 0.7676542802385402, 4.677810902896065), (1000, 3): (677, 1428.2028072016205, -2.7978472945909423), (5, 2): (3, -4.19216453734978, 7.440085041919248), (6, 2): (3, 2.567400843704508, 3.865400737719117), (7, 2): (3, 5.301708084743173, 2.584456386779305), (8, 2): (3, 10.585846370666784, 0.2132226812301181), (9, 2): (3, 4.743682340477754, 2.469742591800006), (10, 2): (3, 1.7376424142176337, 3.591506322956243), (11, 2): (3, 4.198590121062746, 2.7236432915584263), (12, 2): (3, 2.158850158270758, 3.4270542226917704), (13, 2): (3, 1.2133116711109333, 3.7417295920173252), (14, 2): (6, -8.590953993819292, 5.63616967258245), (15, 2): (3, 0.3380267286949867, 4.021751908164715), (16, 2): (3, 0.7786622394396139, 3.8936213541501816), (17, 2): (3, 0.656855947084955, 3.9229802177143425), (18, 2): (3, 0.9492696874148265, 3.8574059219109156), (19, 2): (3, -0.25027328880443134, 4.119681604684494), (20, 2): (3, 0.25561428340377135, 4.013671871172086), (21, 2): (3, 0.3598802796

In [74]:
# The best choice for splitting point when j=1000 and k=3 is stored in first_split.
# Thus, the first line will cover points in (first_split, 1000)
first_split = best_choice[1000,3]
first_split

677

In [75]:
# We now consider j=677-1=676 and k=3-1=2
# The best choice for splitting point given the above j,k is stored in second_split.
# Thus, the second line will cover points in (second_split, 676).
# Since we only have three lines, the third will cover points in (1, second_split-1)
second_split = best_choice[first_split-1,2]
second_split

324

In [76]:
# Hence, to obtain the coefficients for each line:
print(f'The third line has coefficients a = {a(677, 1000)}, b = {b(677, 1000)}')
print(f'The second line has coefficients a = {a(324, 676)}, b = {b(324, 676)}')
print(f'The first line has coefficients a = {a(1, 323)}, b = {b(1, 323)}')

The third line has coefficients a = 1428.2028072016205, b = -2.7978472945909423
The second line has coefficients a = 499.52630522715805, b = -0.08277141390232377
The first line has coefficients a = 41.41697391132595, b = 2.6198719328315083


b)

In [125]:
# Here, we also need a cost per line C
C = int(input())

f = [None] * (n+1)
best_choice = dict()
    
# base case
f[0] = 0

for j in range(1, n+1):
    answer = float('inf')
    
    # Find the minimum penalty per j
    for i in range(1,j):
        penalty = L(i,j) + C + f[i-1]
        
        if penalty < answer:
            answer = penalty
            
            # best_choice[j] refers to the optimal split given specific j
            best_choice[j] = i
    
    f[j] = answer

print(f[n])

10000
100109.20639497414


In [90]:
print(best_choice[1000])

841


In [99]:
print(f'The 6th line will have coefficients a={a(841,1000)} and b={b(841,1000)}.')

The 6th line will have coefficients a=1693.969139518941 and b=-3.3706682936738726.


In [91]:
print(best_choice[840])

677


In [100]:
print(f'The 5th line will have coefficients a={a(677,840)} and b={b(677,840)}.')

The 5th line will have coefficients a=1182.461368158822 and b=-2.1598034478458956.


In [92]:
best_choice[676]

497

In [101]:
print(f'The 4th line will have coefficients a={a(497,676)} and b={b(497,676)}.')

The 4th line will have coefficients a=704.0776964268915 and b=-0.767198281558675.


In [93]:
best_choice[496]

313

In [102]:
print(f'The 3rd line will have coefficients a={a(313,496)} and b={b(313,496)}.')

The 3rd line will have coefficients a=334.1239676269403 and b=0.7051310552970829.


In [95]:
best_choice[312]

157

In [103]:
print(f'The 2nd line will have coefficients a={a(157,312)} and b={b(157,312)}.')

The 2nd line will have coefficients a=125.49428628367815 and b=1.975571957171812.


In [96]:
best_choice[156]

1

In [104]:
print(f'The 1st line will have coefficients a={a(1,156)} and b={b(1,156)}.')

The 1st line will have coefficients a=11.47512592259059 and b=3.2698572772406687.


### #3

a)

In [133]:
n = int(input())
w = [0] + [int(x) for x in input().split()]
M = int(input())

# Let f(i, M1, M2, M3) be whether the i-th trash bag can still fit inside one of the containers
# given their current capacities M1, M2, and M3

memo = dict()
def f(i, M1, M2, M3):
    if (i, M1, M2, M3) in memo:
        return memo[i, M1, M2, M3]
    else:
        if i == n:
            
            # If the all containers still have capacity left, and if the last trash bag
            # can fit in one of them, then we know that all trash bags will fit in all containers
            if M1 >= 0 and M2 >= 0 and M3 >= 0 and (M1 - w[i] >= 0 or M2 - w[i] >= 0 or M3 - w[i] >= 0):
                answer = True
            else:
                answer = False
        else:
            # We can place it in container 1, container 2, or container 3
            # The checking of whether it fits only happens in the base case
            answer = f(i+1, M1-w[i], M2, M3) or f(i+1, M1, M2-w[i], M3) or f(i+1, M1, M2, M3-w[i])
        
        memo[i,M1,M2,M3] = answer
        return memo[i,M1,M2,M3]
    
print(f(1, M, M, M))

8
3 2 12 7 1 6 2 6
13
True


b)

In [78]:
n = int(input())
w = [0] + [int(x) for x in input().split()]
M = int(input())

# We get the sum of the weights of all trash bags.
# This denotes the total weight of trash bags not in container 1
total_w = sum(w)

# Let f(i, M1) be whether the i-th trash bag can still fit inside one of the containers
# given that the first one has current capacity M1

memo = dict()
def f(i, M1):
    if (i, M1) in memo:
        return memo[i, M1]
    else:
        if i == n:
            
            # If the last trash bag fits in container 1 and
            # the total weight of the discarded trash bags is less than or equal to M,
            # then all trash bags fit in the two containers
            # (we can put the "discarded" trash bags into container 2)
            if M1 - w[i] >= 0 and total_w - (M - M1 + w[i]) <= M:
                answer = True
            
            # We cannot place the last trash bag in container 1.
            # If the total weight of bags outside container 1 is less than or equal` to M,
            # then all trash bags still fit in the two containers
            elif M1 - w[i] < 0 and total_w - (M - M1) <= M:
                answer = True
            else:
                answer = False
        else:
            
            # If the current trash bag fits in container 1,
            # then we can either place it in container 1, reducing its capacity,
            # and move to the next trash bag, or not place it in container 1.
            if M1 - w[i] >= 0:
                answer = f(i+1, M1-w[i]) or f(i+1, M1)
            
            # If the current trash bag does not fit in container 1,
            # then we move to the next trash bag, and
            # the total weight of bags not in container 1 remains the same
            else:
                answer = f(i+1, M1)
                
    
        memo[i, M1] = answer
        return memo[i, M1]
    
print(f(1, M))

5
1 2 3 4 5
8
True


### #4

In [2]:
# we assume that the array is zero-indexed
A = [int(x) for x in input().split()]
X = int(input())

# we consider the prefix-sum array of A
P = []
running_sum = 0
for x in A:
    running_sum += x
    P.append(running_sum)

memo = dict()

# Ls record the left indices for the maximum subarray sum of each R
Ls = dict()

# Let f(R) be the maximum subarray sum among all subarrays ending in R
def f(R):
    if R in memo:
        return memo[R]
    else:
        
        # If the subarray ends before X-1, that is, if it has less than X elements,
        # then we disregard it
        if R < X - 1:
            answer = 0
            
        # The maximum subarray sum for the subarray starting at 0 and ending at X-1
        # is just P[R] = P[X-1].
        # The left index is then 0
        elif R == X - 1:
            answer = P[R]
            Ls[R] = 0
        
        
        else:
            
            # Similar to the original maximum subarray sum problem, we can use f(R-1)
            # to compute for f(R).
            # However, in this case, A[R] is replaced by the sum of the subarray
            # with exactly X elements and ending in R
            old_max = A[R] + f(R-1)
            new_val = P[R] - P[R-X]
            
            # If the old maximum is still the maximum, we retain its left index
            if old_max >= new_val:
                answer = old_max
                Ls[R] = Ls[R-1]
            
            # If not, the new left index is the left index of the subarray with X elements
            # ending in R
            elif new_val > old_max:
                answer = new_val
                Ls[R] = R-X+1
            
        memo[R] = answer
        return memo[R]
    
n = len(A)
f(n-1)

R = max(memo, key=memo.get)
print(f'The maximum subarray sum is {max(memo.values())}, where L = {Ls[R]} and R = {R}.')

 1 -2 -3 10 -4 7 2 -5
 5


The maximum subarray sum is 12, where L = 2 and R = 6.


### #5

In [135]:
A = input()
B = input()
C = input()

m = len(A)
n = len(B)
m_n = len(C)

memo = dict()

# Let f(a, b, c) be the number of ways that C with current length c is a shuffle 
# of A with current length a and B with current length b
def f(a, b, c):
    if (a,b,c) in memo:
        return memo[a,b,c]
    else:
        
        # Base case: we only have the first character of C remaining
        if c == 1:
            
            # If the first character of C is not the first character of A nor B,
            # then C is not a shuffle of A and B
            if A[0] != C[0] and B[0] != C[0]:
                answer = 0
            elif A[0] == C[0] or B[0] == C[0]:
                answer = 1
        else:
            
            # We first let answer be 0.
            # Given the two if statements below, this means that we first consider
            # the case when the current character of C is not equal to that of A and B
            answer = 0
            
            # If the current character of C is the same as the current character of A,
            # then assign that character to A and move to the next character.
            # We also check if the length of A is greater than 0 to make sure it does not go back to the last character
            if a > 0 and A[a-1] == C[c-1]:
                answer = f(a-1, b, c-1)
                
            # Same as the above but with B instead of A.
            # If both are true, then we add both branches: when we assign the character to A
            # and when we assign the character to B.
            # We also check if the length of B is greater than 0 to make sure it does not go back to the last character
            if b > 0 and B[b-1] == C[c-1]:
                answer += f(a, b-1, c-1)
        
        memo[a,b,c] = answer
        return memo[a,b,c]
    
print(f(m, n, m_n))

AB
AB
ABAB
2
